In [1]:
from agents import *
from notebook import psource


In [ ]:
'''
In this context what we examine is the taxi driver world.Here the taxi driver is supposed to be driving in pre determined
pattern {zig zag} , find a customer and drop him to the desired location.
There are some more variables such as fuel levels and cab occupancy.

The problem can be formulated as follows:

• Initial state: Any state can be designated as the initial state however i have assumed it to be [0,0] 
• Actions: In this simple environment, each state has just following  actions: turn Left or  Right if a bump has been detected, move forward , pickup and drop a customer and fill petrol.
• Goal test: This whether all the existing customers have been serviced given adequate fuel levels. 
• Path cost: Each step costs 5 units of fuel. The agent would die if it runs out of fuel. 

''' 

In [2]:
class TheDumbTaxiDriver(Agent):
    current_cust_drop_loc=[2,2]
    is_customer_in=False
    iteration=0
    location=[0,1]
    direction=Direction('right')
    fuel_level=15
    def drive(self,thing):
        if  isinstance(thing,Customer):
            print('The agent is driving')
            return True
        else:
            False
    def hasEnoughFuel(self):
        if (self.fuel_level==0):
            return False
        else:
            return True
        
    def consumeFuel(self):
        self.fuel_level-=5
        return True
    
    def fillup(self,thing):
        if isinstance(thing,Petrol) and (self.fuel_level<20):
        
            print('Petrol low!!The agent is filling up gas..fuel level:{}'.format(self.fuel_level))
            self.fuel_level=100
            return True
        else:
            print('Petrol enough!!The agent decided not to fill up gas..fuel_level:{}'.format(self.fuel_level))
            return False
        
        return False
    
    def fillup(self):
        if (self.fuel_level<20):
        
            print('Petrol low!!The agent is filling up gas..fuel level:{}'.format(self.fuel_level))
            self.fuel_level=100
            return True
        else:
            print('Petrol enough!!The agent decided not to fill up gas..fuel_level:{}'.format(self.fuel_level))
            return False
        
        return False
    
    ''''def movedown(self):
        self.location += 1''' 
    
    def turn(self,d):
        print('Inside turn..turning {}'.format(d))
        self.direction= self.direction+d
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        self.consumeFuel()
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def findNearestPump(self,pump_locations):
        lowestcost=1000
        lowest_cost_location=[]
        for pump_location in pump_locations:
            count=self.findDistance(self.location,pump_location)
            if(count<lowestcost):
                lowestcost=count
                lowest_cost_location=pump_location
        print('The Lowest Count is :{0} for location :{1} '.format(lowestcost,lowest_cost_location))
        return lowest_cost_location
        
    def findDistance(self,current_location,to_location):
        x1=current_location[0]
        y1=current_location[1]
        x2=to_location[0]
        y2=to_location[1]
        count=0
        num_steps=0
#         print('Count:',count)
        for x in range(abs(y2-y1)):   
            if(y2>y1):
                print('a')

                if   (x2>x1):
                    #print('aa')
                    x1+=1 
                    y1+=1
                    count+=1

                elif (x2<x1):
                    #print('ab')
                    x1-=1
                    y1+=1
                    count+=1


            elif (y2<y1):
#                 print('b')

                if   (x2>x1):
#                     print('ba')
                    y1-=1
                    x1+=1

                elif (x2<x1):
#                     print('bb')
                    y1-=1
                    x1-=1
                    count+=1

                else:
#                     print('bc')
                    pass


        if(y1!=y2):
            num_steps=abs(y1-y2)+1
        elif(x1!=x2):
            num_steps=abs(x1-x2)+1



        for x in range(num_steps):
            if   x2>x1:
                x1+=1
                count+=1

            elif x2<x1:
                x1-=1
                count+=1

        for z in range(num_steps):
            if(y2>y1):
                y1+=1
                count+=1
                 

            elif(y2<y1):
                y1-=1
                count=count+1
                 
        
        return count





In [3]:
class Customer(Thing):
    def __init__(self,drop_location=[1,1]):
        self.drop_location=drop_location
    
class Petrol(Thing):
    pass
class Bump(Thing):
    pass

class DropLocation(Thing):
    drop_location=[]
    pass

class City(GraphicEnvironment):
    clues={
               'customers':[],
               'petrol_pumps':[]
              }
    def percept(self, agent):
        things=self.list_things_at(agent.location)
        
        
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
            
        return things
    
    def lookForCustomers(self, agent):
        things=self.list_things_at(agent.location)
        col=self.width
        rows=self.height
        for x in range(col):
            for y in range(rows):                
                customers=self.list_things_at([x,y],tclass=Customer)
                
                if(len(customers)!= 0):
                    for customer in customers:
                        self.clues['customers'].append([x,y])
                        print('Found and loaded the customer in the list at loc: {0},{1}'.format(x,y))
                
                petrols=self.list_things_at([x,y],tclass=Petrol)
                
                if(len(petrols)!= 0):
                    for petrol in petrols:
                        self.clues['petrol_pumps'].append([x,y])
                        print('Found and loaded the petrol in the list at loc: {0},{1}'.format(x,y))
                
        
        return True
    
    def findPath(self,agent,current_location,to_location):
        print('type of self: ',type(self))
        
        
        print('The current_location:{0} , to_location:{1} , fuel level:{2}'.format(agent.location,to_location,agent.fuel_level))
        
        [x1,y1]=current_location        
        [x2,y2]=to_location        
        num_steps=0
        
        for x in range(1, abs(y2-y1)+1 ):
            if not(agent.hasEnoughFuel()):
                return False
            print('Hello ',agent.location)
            if(y2>agent.location[1]):

                if   ( x2>agent.location[0] ):
                    agent.location[1]+=1
                    agent.location[0]+=1 
                    agent.consumeFuel()
                    self.reveal()
                    
                elif ( x2<agent.location[0] ):
                    agent.location[1]+=1
                    agent.location[0]-=1 
                    agent.consumeFuel()
                    self.reveal()
                    
                print('1.The current_location:{0} , to_location:{1} , fuel level:{2}'.format(agent.location,to_location,agent.fuel_level))
                
            elif ( y2<agent.location[1] ):
                
                if   ( x2>agent.location[0] ):
                    agent.location[1]-=1
                    agent.location[0]+=1
                    agent.consumeFuel()
                    self.reveal()
                    
                elif ( x2<agent.location[0] ):
                    
                    agent.location[1]-=1
                    agent.location[0]-=1
                    agent.consumeFuel()
                    self.reveal()
            
                print('2.The current_location:{0} , to_location:{1} , fuel level:{2}'.format(agent.location,to_location,agent.fuel_level))
            else:
                pass
            
        if  (agent.location[0]!=x2):
            num_steps=abs(agent.location[0]-x2)+1
        elif(agent.location[1]!=y2):
            num_steps=abs(agent.location[1]-y2)+1            
            
        for x in range(1, num_steps ):
            if not(agent.hasEnoughFuel()):
                return False
            
            elif (x2>agent.location[0]):
                agent.location[0]+=1
                agent.consumeFuel()
                self.reveal()
                print('3.The current_location:{0} , to_location:{1} , fuel level:{2}'.format(agent.location,to_location,agent.fuel_level))
            elif (x2<agent.location[0]):
                agent.location[0]-=1
                agent.consumeFuel()
                self.reveal()
                print('4.The current_location:{0} , to_location:{1} , fuel level:{2}'.format(agent.location,to_location,agent.fuel_level))
            else:
                pass
            
            
            
        for y in range(1,num_steps):
            if not(agent.hasEnoughFuel()):
                return False
            
            elif (y2>agent.location[1]):
                agent.location[1]+=1
                self.reveal()
                print('5.The current_location:{0} , to_location:{1} , fuel level:{2}'.format(agent.location,to_location,agent.fuel_level))
            elif (y2<agent.location[1]):
                agent.location[1]-=1
                agent.consumeFuel()
                self.reveal()
                print('6.The current_location:{0} , to_location:{1} , fuel level:{2}'.format(agent.location,to_location,agent.fuel_level))
            else:
                pass
            
            
        if agent.location==to_location:
            print('Huuray!! Found my customer or dropped customer!!')
            return True
                
        
        
    
        
    
    def execute_action(self, agent, action):
        print('The width :{0} and height :{1} of the matrix is'.format(self.width,self.height))
        print('Inside Execute Action for Iteration:{0}  and action:{1} with direction:{2} and fuel level:{3}'.format(agent.iteration,action,agent.direction.direction,agent.fuel_level) )
        agent.iteration += 1
        self.lookForCustomers(agent)
        print('List of Customers:{0}  and list of petrol pumps:{1}'.format(self.clues['customers'],self.clues['petrol_pumps']))
        if(agent.fuel_level<=20):
            nearest_pump_loc=agent.findNearestPump(self.clues['petrol_pumps'])
            self.findPath(agent,agent.location,nearest_pump_loc)
            agent.fillup()
        if(len(self.clues['customers'])!=0):    
            cust_pickup_location=self.clues['customers'][0]
            things=self.list_things_at(cust_pickup_location,tclass=Customer)
            cust_drop_location=things[0].drop_location

            if( self.findPath(agent,agent.location,cust_pickup_location) ):
                self.delete_thing(things[0])

                if (self.findPath(agent,agent.location,cust_drop_location) ):
                    self.clues['customers'].pop(0)                            
                    print('The remaining list of customers with the driver:{} '.format(self.clues['customers'])  )             
                else:
                    agent.alive=False            
                    print('Agent has run out of gas.Agent is dead.')

            else:
                agent.alive=False            
                print('Agent has run out of gas.Agent is dead.')
        else:
            self.is_done()
        return action
    
    
    def is_done(self):
        out_of_fuel=False
        no_edibles = not any(isinstance(thing, Customer)  for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        for agent in self.agents:
            if(agent.fuel_level==0):               
                out_of_fuel=True
        if(no_edibles):
            print('No more work  today..Let"s rest')
        return no_edibles or dead_agents or out_of_fuel
    

In [4]:
def program(percepts):
    
    for percept in percepts:
        
        if isinstance(percept,Customer):
            
            return 'pickup'
        elif isinstance(percept,Petrol):
            return 'fillup'
        elif isinstance(percept,Bump):
            
            return 'turn'

    
    return 'moveforward'


        
    

In [5]:
city = City(10,5, color={'TheDumbTaxiDriver': (200,0,0), 'DropLocation':(200,100,165), 'Customer': (0, 200, 200), 'Petrol': (230, 115, 40)}) # park width is set to 5, and height to 20
theDumbTaxiDriver = TheDumbTaxiDriver(program)

customer = Customer(drop_location=[2,1])
customer2 = Customer(drop_location=[7,2])
customer3 = Customer(drop_location=[0,0])
drop_location3=DropLocation()
drop_location2=DropLocation()
petrol = Petrol()
more_petrol=Petrol()
city.add_thing(theDumbTaxiDriver,[0,4])
city.add_thing(customer3,[6,4])
city.add_thing(drop_location3,[0,0])
city.add_thing(drop_location2,[7,2])
city.add_thing(customer2,[5,2])
#city.add_thing(customer,[6,0])
city.add_thing(petrol,[0,1])
city.add_thing(more_petrol,[1,0])

city.run(100)



,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


The width :10 and height :5 of the matrix is
Inside Execute Action for Iteration:0  and action:moveforward with direction:right and fuel level:15
Found and loaded the petrol in the list at loc: 0,1
Found and loaded the petrol in the list at loc: 1,0
Found and loaded the customer in the list at loc: 5,2
Found and loaded the customer in the list at loc: 6,4
List of Customers:[[5, 2], [6, 4]]  and list of petrol pumps:[[0, 1], [1, 0]]
The Lowest Count is :3 for location :[0, 1] 
type of self:  <class '__main__.City'>
The current_location:[0, 4] , to_location:[0, 1] , fuel level:15
Hello  [0, 4]
2.The current_location:[0, 4] , to_location:[0, 1] , fuel level:15
Hello  [0, 4]
2.The current_location:[0, 4] , to_location:[0, 1] , fuel level:15
Hello  [0, 4]
2.The current_location:[0, 4] , to_location:[0, 1] , fuel level:15


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


6.The current_location:[0, 3] , to_location:[0, 1] , fuel level:10


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


...

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


4.The current_location:[0, 0] , to_location:[0, 0] , fuel level:30
Huuray!! Found my customer or dropped customer!!
The remaining list of customers with the driver:[[6, 4]] 


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


No more work  today..Let"s rest


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


In [6]:
def findDistance(current_location,to_location):
        
        [x1,y1]=current_location       
        [x2,y2]=to_location
        #x1=0   y1=4   x2=1  y2=0        
        #x1=1   y1=3   x2=1  y2=0
        
        #x1=0   y1=4   x2=1  y2=0
        #x1=0   y1=4   x2=1  y2=0
        count=0
        print('x1:{}  y1:{}'.format(x1,y1))
      
        
        num_steps=0
        print('Count:',count)
        for x in range(1, 4):   
            if(y2>y1):
                y1+=1
                if   (x2>x1):
                    x1+=1 
                    count+=1
                    print('1 Count:',count)
                    print('x1:{}  y1:{}'.format(x1,y1))
                elif (x2<x1):
                    x1-=1
                    count+=1
                    print('2 Count:',count)
                    print('x1:{}  y1:{}'.format(x1,y1))
             
            elif (y2<y1):
                y1-=1
                if   (x2>x1):
                    x1+=1
                    count+=1
                    print('3 Count:',count)
                    print('x1:{}  y1:{}'.format(x1,y1))
                elif x2<x1:
                    x1-=1
                    count+=1
                    print('4 Count:',count)
                    print('x1:{}  y1:{}'.format(x1,y1))
            else:
                pass
            
        if  (x1!=x2):
            num_steps=abs(x1-x2)+1
        elif(y1!=y2):
            num_steps=abs(y1-y2)+1 
            
        print('Number of steps:',num_steps)
            
        for x in range(1,num_steps):
            if   x2>x1:
                x1+=1
                count+=1
                print('5 Count:',count)
                print('x1:{}  y1:{}'.format(x1,y1))
            elif x2<x1:
                x1-=1
                count+=1
                print('6 Count:',count)
                print('x1:{}  y1:{}'.format(x1,y1))
            else:
                pass
        for z in range(1,num_steps):
            if   (y2>y1):
                y1+=1
                count+=1
                print('7 Count:',count)
                print('x1:{}  y1:{}'.format(x1,y1))
            elif (y2<y1):
                y1-=1
                count+=1
                print('8 Count:',count)
                print('x1:{}  y1:{}'.format(x1,y1))
            else:
                pass
        print('Huuray!! Found my customer !!')
        return count

In [7]:
count=findDistance([0,4],[1,0])
print('Count is :',count)

accahsdadjhadhjcgf
x1:0  y1:4
Count: 0
3 Count: 1
x1:1  y1:3
Number of steps: 2
8 Count: 2
x1:1  y1:0
Huuray!! Found my customer !!
Count is : 2
